# oak-d × spectacularAI × nerfstudio Google Colab ワークフロー

このノートブックは、oak-dで撮影したデータ（spectacularAI形式）をGoogle Colabにアップロードし、nerfstudioで学習・可視化・ダウンロードするまでの一連の手順をまとめたものです。

---

## 概要
1. **Colab環境セットアップ** - PyTorch(GPU対応)のインストールとGPU詳細確認
2. **データアップロード** - processed.zipをアップロード・展開・パス修正
3. **nerfstudioインストール** - nerfstudioのインストールと動作確認
4. **データ確認** - アップロードしたデータの詳細構造確認
5. **学習実行** - nerfstudioでの学習とGPU使用状況監視・進捗表示
6. **結果ダウンロード** - 学習済みモデル(.ckpt)とconfig.ymlのダウンロード

## 前提条件
- ローカルで `sai-cli process --format nerfstudio` により processed/ フォルダが作成済み
- processed.zip ファイルが準備済み

## 主な改善点
- 🔧 **Windows互換性**: パス区切り文字の自動修正機能
- 📊 **GPU監視**: リアルタイムGPUメモリ使用量表示
- ⚡ **エラーハンドリング**: 各段階での詳細なエラー診断
- 💾 **ファイル管理**: サイズ確認付きの選択的ダウンロード
- ⏱️ **進捗表示**: 経過時間とステップ数の監視

In [ ]:
# 1. Colab環境セットアップ
print("=== Python・GPU環境確認 ===")
!python --version
!nvidia-smi  # GPU確認

print("\n=== パッケージインストール ===")
!pip install --upgrade pip

# GPU対応PyTorchのインストール
print("PyTorch(GPU対応)をインストール中...")
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# nerfstudioのインストール
print("nerfstudioをインストール中...")
!pip install nerfstudio

print("\n=== GPU利用可能性チェック ===")
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU device: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    print(f"CUDA version: {torch.version.cuda}")
    print("✅ GPU学習が利用可能です")
else:
    print("❌ GPUが利用できません（CPUで学習されます）")
    print("💡 ランタイム → ランタイムのタイプを変更 → GPU(T4)を選択してください")

In [ ]:
# 2. データをGoogle Colabにアップロード
# processed/ フォルダをzipファイルにしてアップロードしてください
# ローカルで: processed フォルダを右クリック → 圧縮 → processed.zip

from google.colab import files
import zipfile
import os
import shutil
import json

print("=== データアップロード ===")
print("processed.zip ファイルをアップロードしてください（NerfStudio形式のデータ）")
uploaded = files.upload()

if not uploaded:
    print("❌ ファイルがアップロードされませんでした")
    exit()

# zipファイルを展開
for filename in uploaded.keys():
    if filename.endswith('.zip'):
        print(f"📦 {filename} を展開中...")
        try:
            with zipfile.ZipFile(filename, 'r') as zip_ref:
                zip_ref.extractall('.')
            print(f"✅ {filename} を展開しました")
            os.remove(filename)  # zipファイルを削除
        except Exception as e:
            print(f"❌ 展開エラー: {e}")
            continue

# Windowsパス区切り文字の問題を修正
print("\n=== パス区切り文字修正 ===")
if os.path.exists('processed'):
    # transforms.jsonのパス区切り文字を修正
    transforms_path = 'processed/transforms.json'
    if os.path.exists(transforms_path):
        try:
            with open(transforms_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
            
            path_fixed = False
            # すべてのパスを修正
            if 'frames' in data:
                for frame in data['frames']:
                    if 'file_path' in frame and '\\' in frame['file_path']:
                        # バックスラッシュをスラッシュに変換
                        frame['file_path'] = frame['file_path'].replace('\\', '/')
                        path_fixed = True
            
            if path_fixed:
                # 修正したJSONを保存
                with open(transforms_path, 'w', encoding='utf-8') as f:
                    json.dump(data, f, indent=2)
                print("✅ transforms.json のパスを修正しました")
            else:
                print("✓ transforms.json のパスは正常です")
                
        except Exception as e:
            print(f"❌ パス修正エラー: {e}")
    else:
        print("⚠️ transforms.json が見つかりません")
else:
    print("❌ processed/ フォルダが見つかりません")

# アップロードされたディレクトリ構造を確認
print("\n=== アップロード確認 ===")
!ls -la
if os.path.exists('processed'):
    print("\n=== processed/ 内容確認 ===")
    !ls -la processed/

In [ ]:
# 3. nerfstudio動作確認
print("=== nerfstudio動作確認 ===")

# nerfstudioのインポートテスト
import nerfstudio
print("✓ nerfstudio が正常にインポートできました")

# バージョン確認
!pip show nerfstudio | grep Version

# コマンドライン確認
print("\n=== nerfstudioコマンド確認 ===")
!ns-train --help | head -5

In [ ]:
# 4. アップロードしたデータの確認
print("=== アップロードデータ構造確認 ===")
!find processed -type f | head -20

print("\n=== 必須ファイル確認 ===")
import os
import json

# transforms.jsonの存在確認と内容確認
if os.path.exists('processed/transforms.json'):
    print("✓ transforms.json が見つかりました")
    
    # transforms.jsonの内容確認
    try:
        with open('processed/transforms.json', 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        if 'frames' in data:
            frame_count = len(data['frames'])
            print(f"  フレーム数: {frame_count}")
            
            # サンプルファイルパスを確認
            if frame_count > 0:
                sample_path = data['frames'][0].get('file_path', '')
                print(f"  サンプルパス: {sample_path}")
                
                # パス形式の確認
                if '\\' in sample_path:
                    print("  ⚠️ Windowsパス区切り文字が残っています（要修正）")
                else:
                    print("  ✓ パス形式は正常です")
        else:
            print("  ❌ framesデータが見つかりません")
            
    except Exception as e:
        print(f"  ❌ transforms.json読み込みエラー: {e}")
else:
    print("❌ transforms.json が見つかりません")

# 画像フォルダの確認
if os.path.exists('processed/images'):
    all_files = os.listdir('processed/images')
    jpg_files = [f for f in all_files if f.lower().endswith(('.jpg', '.jpeg'))]
    png_files = [f for f in all_files if f.lower().endswith('.png')]
    
    print(f"✓ 画像フォルダに {len(jpg_files)} 枚のJPG画像と {len(png_files)} 枚のPNG画像があります")
    
    if jpg_files:
        print(f"  サンプル画像: {jpg_files[0]}")
    
    total_images = len(jpg_files) + len(png_files)
    if total_images > 0:
        print("✅ 学習準備完了！")
        if total_images < 10:
            print("⚠️ 画像数が少ないため、学習結果の品質が低い可能性があります")
    else:
        print("❌ 画像ファイルがありません")
else:
    print("❌ images フォルダが見つかりません")

# その他の重要ファイル確認
additional_files = ['processed/sparse_pc.ply', 'processed/colmap']
for file_path in additional_files:
    if os.path.exists(file_path):
        print(f"✓ {file_path} が見つかりました")
    else:
        print(f"  {file_path} が見つかりません（オプション）")

In [ ]:
# 5. Nerfstudio学習＋リアルタイムログ表示＋中断時保存
import subprocess
import sys
import os
import time
import threading

print("=== 学習開始 ===")
print("⚠️ 学習には数分〜数十分かかります")
print("💡 Ctrl+C で中断可能です（途中結果は保存されます）\n")

# GPU監視スレッド
def monitor_gpu():
    """GPUメモリ使用量を定期的に表示"""
    import torch
    if torch.cuda.is_available():
        while True:
            try:
                memory_used = torch.cuda.memory_allocated(0) / 1024**3
                memory_total = torch.cuda.get_device_properties(0).total_memory / 1024**3
                print(f"📊 GPU Memory: {memory_used:.1f}/{memory_total:.1f} GB", end='\r')
                time.sleep(30)  # 30秒ごとに更新
            except:
                break

# 学習コマンド
cmd = [
    "ns-train",
    "nerfacto",
    "--pipeline.datamanager.data", "processed/",
    "--output-dir", "outputs/",
    "--max-num-iterations", "30000",         # 最大イテレーション
    "--steps-per-save", "500",               # 500ステップごとに自動保存
    "--logging.steps-per-log", "10"          # ログ表示間隔
]

print(f"実行コマンド: {' '.join(cmd)}\n")

# GPU監視開始
import torch
if torch.cuda.is_available():
    monitor_thread = threading.Thread(target=monitor_gpu, daemon=True)
    monitor_thread.start()

start_time = time.time()
process = subprocess.Popen(
    cmd,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    universal_newlines=True,
    bufsize=1
)

try:
    line_count = 0
    for line in process.stdout:
        print(line, end="")
        sys.stdout.flush()
        line_count += 1
        
        # 100行ごとに経過時間を表示
        if line_count % 100 == 0:
            elapsed = time.time() - start_time
            print(f"\n⏱️ 経過時間: {elapsed/60:.1f}分")
            
except KeyboardInterrupt:
    print("\n⚠️ 学習を中断しました")
    # プロセス終了
    process.terminate()
    process.wait()
    
    # 最新のチェックポイントを確認
    print("💾 途中までの学習結果を確認中...")
    if os.path.exists("outputs"):
        import glob
        ckpt_files = glob.glob('outputs/**/*.ckpt', recursive=True)
        if ckpt_files:
            print(f"✅ {len(ckpt_files)} 個のチェックポイントが保存されています")
        else:
            print("❌ チェックポイントファイルが見つかりません")

process.wait()
elapsed_total = time.time() - start_time

print(f"\n✅ 学習コマンドが完了しました")
print(f"⏱️ 総学習時間: {elapsed_total/60:.1f}分")
print("📂 出力先: outputs/")

# 学習結果の簡易確認
if os.path.exists("outputs"):
    import glob
    ckpt_files = glob.glob('outputs/**/*.ckpt', recursive=True)
    config_files = glob.glob('outputs/**/config.yml', recursive=True)
    print(f"📊 生成ファイル: .ckpt={len(ckpt_files)}個, config.yml={len(config_files)}個")

In [ ]:
# 6. 学習結果のダウンロード
print("=== 学習結果のダウンロード ===")
print("💾 ファイルはブラウザのダウンロードフォルダに保存されます")

from google.colab import files
import glob
import os

# outputs/ ディレクトリの存在確認
if not os.path.exists('outputs'):
    print("❌ outputs/ フォルダが見つかりません")
    print("💡 学習が正常に完了していない可能性があります")
    exit()

# outputs/ ディレクトリ内の全体構造を確認
print("\n📁 学習結果のディレクトリ構造:")
!find outputs -type f | head -20

# 重要ファイルの確認と統計
ckpt_files = glob.glob('outputs/**/*.ckpt', recursive=True)
config_files = glob.glob('outputs/**/config.yml', recursive=True)
json_files = glob.glob('outputs/**/*.json', recursive=True)

print(f"\n📊 ファイル統計:")
print(f"  チェックポイント(.ckpt): {len(ckpt_files)} 個")
print(f"  設定ファイル(.yml): {len(config_files)} 個")
print(f"  JSONファイル(.json): {len(json_files)} 個")

# ファイルサイズの表示
def get_file_size(filepath):
    """ファイルサイズをMB単位で取得"""
    try:
        size_bytes = os.path.getsize(filepath)
        return size_bytes / (1024 * 1024)  # MB
    except:
        return 0

# チェックポイントファイルのダウンロード
if ckpt_files:
    print(f"\n📥 学習済みモデル(.ckpt)をダウンロード中...")
    for file in ckpt_files:
        if os.path.isfile(file):
            size_mb = get_file_size(file)
            print(f"  ダウンロード: {os.path.basename(file)} ({size_mb:.1f} MB)")
            try:
                files.download(file)
            except Exception as e:
                print(f"    ❌ ダウンロードエラー: {e}")
else:
    print("\n❌ .ckptファイルが見つかりませんでした")
    print("💡 学習が完了していない可能性があります")

# 設定ファイルのダウンロード
if config_files:
    print(f"\n📥 設定ファイル(config.yml)をダウンロード中...")
    for file in config_files:
        if os.path.isfile(file):
            size_mb = get_file_size(file)
            print(f"  ダウンロード: {os.path.basename(file)} ({size_mb:.2f} MB)")
            try:
                files.download(file)
            except Exception as e:
                print(f"    ❌ ダウンロードエラー: {e}")

# JSONファイルのダウンロード（サイズ制限あり）
if json_files:
    print(f"\n📥 JSONファイルをダウンロード中...")
    downloaded_count = 0
    for file in json_files:
        if downloaded_count >= 5:  # 最大5個まで
            break
        if os.path.isfile(file):
            size_mb = get_file_size(file)
            if size_mb < 10:  # 10MB未満のファイルのみ
                print(f"  ダウンロード: {os.path.basename(file)} ({size_mb:.2f} MB)")
                try:
                    files.download(file)
                    downloaded_count += 1
                except Exception as e:
                    print(f"    ❌ ダウンロードエラー: {e}")
            else:
                print(f"  スキップ: {os.path.basename(file)} (サイズ: {size_mb:.1f} MB - 大きすぎます)")

print(f"\n✅ ダウンロード完了！")
print(f"💡 ローカルでnerfstudio viewerを使って結果を確認できます")
print(f"🚀 ローカル確認コマンド: ns-viewer --load-config path/to/config.yml")